In [1]:
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
from datetime import datetime, date
import json
from xgrads import open_CtlDataset
import geopandas as gpd
import requests
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube
%matplotlib inline

# In QGIS

Once you have the huc4 geojson, the streamgage station shapefiles and the filled dem downloaded - you need to do some work in QGIS to determine the upslope area of each stream gage. 

A detailed description of the workflow can be found here: https://docs.google.com/document/d/1Urdpp41tCZp0ZptHDdRD2HQ3ZznolhWA0Zaa_szcgfc/edit?usp=sharing
Here is the overview:
1. Load DEM, huc4 shapefile, and gage shapefile
2. Reproject huc4 shape to match DEM
2. Clip dem to HUC4
3. Fill Sinks tool
4. Strahler Order tool
5. Using station locations, use Upslope Area tool to save out .tif files with upslope area for each gage. 
6. Rasterize the huc4 shapefile to use for masking later. 

In [2]:
# Choose a domain
domain = 'BEAU'

# Path to the SnowModel folder
gagepath = '/nfs/attic/dfh/2020_NPRB/data/stream_gage/'+domain+'/'

#path to NPRB domains
domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
domains = domains_resp.json()

In [3]:
list_of_files = sorted( filter( os.path.isfile,
                        glob.glob(gagepath + '*.tif') ) )

In [4]:
list_of_files

['/nfs/attic/dfh/2020_NPRB/data/stream_gage/CHUK/15565700.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/CHUK/15580095.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/CHUK/15621000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/CHUK/15744500.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/CHUK/15746900.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/CHUK/15746980.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/CHUK/15746990.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/CHUK/15746991.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/CHUK/15747000.tif']

In [5]:
for i in range(len(list_of_files)):

    file = list_of_files[i]
    stncode=file[-12:-4]
    data = rxr.open_rasterio(file)
    da = data.sel(band=1)
    if i ==0:
        ds = da.to_dataset(name=stncode)
    else:
        ds[stncode]=da
ds

<xarray.Dataset>
Dimensions:      (x: 763, y: 798)
Coordinates:
    band         int64 1
  * y            (y) float64 2.158e+06 2.158e+06 ... 1.362e+06 1.362e+06
  * x            (x) float64 -7.435e+05 -7.425e+05 ... 1.75e+04 1.85e+04
    spatial_ref  int64 0
Data variables:
    15565700     (y, x) float32 ...
    15580095     (y, x) float32 ...
    15621000     (y, x) float32 ...
    15744500     (y, x) float32 ...
    15746900     (y, x) float32 ...
    15746980     (y, x) float32 ...
    15746990     (y, x) float32 ...
    15746991     (y, x) float32 ...
    15747000     (y, x) float32 ...

In [6]:
ds.to_netcdf(gagepath+'upslope.nc')